# Target : summarizing the reviews on a given anime

Objective : only have a few sentences summarizing an anime's strongpoints and drawbacks instead of having to browse through many reviews, to get a general impression that is representative of the whole community's feelings.

In [1]:
#Imports

import pandas as pd
import numpy as np
import ast
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import torch
import transformers
import re
import string
import tiktoken
import requests
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")


/home/antog96/Jedha_Bootcamp/Projet_DSFS/DataScience/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda device


In [7]:
tqdm.pandas()

In [3]:
!python -m spacy download en_core_web_sm -q

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
from transformers import pipeline

pipe_summarization = pipeline("summarization", model="facebook/bart-large-cnn",)

pipe_sentiment = pipeline("sentiment-analysis")

Device set to use cuda:0
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


In [2]:
#Dataset import
df_reviews = pd.read_csv("https://anime-recommendation-engine.s3.eu-west-3.amazonaws.com/data/reviews_clean.csv")


In [3]:
df_reviews.head()

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149


In [7]:
reviewed_animes = df_reviews['anime_uid'].nunique()
print(f'Number of animes with a review : {reviewed_animes}')
print(f'Total number of reviews : {len(df_reviews)}')
print(f'Average number of reviews per reviewed anime : {len(df_reviews)/reviewed_animes}')


Number of animes with a review : 8113
Total number of reviews : 130519
Average number of reviews per reviewed anime : 16.08763712560089


In [8]:
#concat_reviews = df_reviews.groupby('anime_uid', as_index=False).agg({'text':' '.join})
#pd.reset_option('display.max_colwidth', None)
#concat_reviews.head(10)

In [9]:
sao_id = 11757

reviews_sao = concat_reviews[concat_reviews['anime_uid'] == sao_id]['text']
print(reviews_sao.apply(len))

NameError: name 'concat_reviews' is not defined

In [9]:
#def lemmatize_text(text):
#    doc = nlp(text)
#    return " ".join([token.lemma_ for token in doc])

#nlp.max_length = 1500000

def clean_text(text):
    text = re.sub("[\n\r]", " ", text)
    text = re.sub("[^A-Za-z0-9]+", " ", text)
    return text

In [ ]:
#concat_reviews["cleaned_text"] = concat_reviews['text'].apply(clean_text)

#cleaned_texts = []

#for i, text in enumerate(concat_reviews['text']):
#    cleaned_text = clean_text(text)
#    cleaned_texts.append(cleaned_text)
#    print(f"Progress: {i + 1}/{len(concat_reviews)}")

#concat_reviews['cleaned_text'] = cleaned_texts

In [13]:
#concat_reviews.head()

In [14]:
#cleaned_texts = []

#for i, text in enumerate(df_reviews['text']):
#    cleaned_text = clean_and_split_text(text)
#    print(f"Progress: {i + 1}/{len(df_reviews)}")

#df_reviews['chunked_text'] = cleaned_texts

In [10]:
reviews_for_llm = df_reviews.drop(columns=["uid","profile","score","scores","link"])
reviews_for_llm['text'] = reviews_for_llm["text"].apply(ast.literal_eval)
reviews_for_llm['text'].progress_apply(clean_text)



IndentationError: unexpected indent (<unknown>, line 9)

In [22]:
df_class.to_csv('cleaned_and_split_reviews.csv')

In [77]:
df_class['chunked_text'][0]

[' more pics Overall Story Animation Sound Character Enjoyment First things first My reviews system is explained on a blog entry Which can be found through my profile I m going to keep this review more of a opinion of Gintama s overall and then this season specific Anyhow What I have always loved regarding Gintama is its content of everything I love the comedy its absurd random can be vile dirty sweet anyhow everything Have laughed countless times in this franchise Also the humor they have also is heavily ref',
 'erence based a k a parodies of different anime shows manga live stuff real world anime production and so on Anyhow comedy parody side of this franchise i absolutely love Now nd side of this show is the serious dramas epic battle shounens and so on There are arcs that are fully comedy arcs that are fully serious and mixtures of both Serious side is usually quite dramatic and managed to somewhat tear me up now and then Whilst the action sequences are absolute bliss as well They 

In [78]:
df_class.loc[:,"classifications"] = df_class["chunked_text"].loc[:50].progress_apply(pipe_sentiment)

100%|██████████| 51/51 [00:06<00:00,  8.36it/s]
/tmp/ipykernel_85088/3745356283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_class.loc[:,"classifications"] = df_class["chunked_text"].loc[:50].progress_apply(pipe_sentiment)


In [67]:
df_class.loc[:,"classifications"] = df_class["chunked_text"].progress_apply(pipe_sentiment)

  3%|▎         | 4002/130519 [11:18<5:57:26,  5.90it/s] 


KeyboardInterrupt: 

In [79]:
df_class["classifications"][3]

[{'label': 'NEGATIVE', 'score': 0.9992042183876038},
 {'label': 'NEGATIVE', 'score': 0.9996769428253174},
 {'label': 'NEGATIVE', 'score': 0.7189896106719971},
 {'label': 'POSITIVE', 'score': 0.5227293968200684},
 {'label': 'POSITIVE', 'score': 0.9850171804428101},
 {'label': 'NEGATIVE', 'score': 0.9953349232673645},
 {'label': 'POSITIVE', 'score': 0.9966551065444946},
 {'label': 'POSITIVE', 'score': 0.9956899285316467},
 {'label': 'NEGATIVE', 'score': 0.9622377753257751},
 {'label': 'NEGATIVE', 'score': 0.9748064279556274},
 {'label': 'POSITIVE', 'score': 0.987274169921875},
 {'label': 'POSITIVE', 'score': 0.8943804502487183}]

In [80]:
def calculate_mean_score(classifications):
    if isinstance(classifications, list):
        total = 0
        count = 0
        for item in classifications:
            score = item['score']
            if item['label'] == 'NEGATIVE':
                score = -score
            total += score
            count += 1
        return total / count if count else 0
    else:
        return 0


In [81]:
df_class.loc[:,'mean_score'] = df_class['classifications'].apply(calculate_mean_score)

/tmp/ipykernel_85088/1758409613.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_class.loc[:,'mean_score'] = df_class['classifications'].apply(calculate_mean_score)


In [82]:
df_class['mean_score'].loc[:10]

0     0.424159
1     0.797101
2     0.332255
3    -0.022375
4     0.506765
5     0.996296
6    -0.059540
7     0.401125
8     0.119885
9    -0.485066
10   -0.985845
Name: mean_score, dtype: float64

In [83]:
def determine_sentiment(score):
    if 0.2 <= score <= 1:
        return "POSITIVE"
    elif -0.2 <= score < 0.2:
        return "MIXED FEELINGS"
    else:
        return "NEGATIVE"

In [84]:
df_class.loc[:,'gen_classification'] = df_class['mean_score'].apply(determine_sentiment)

In [85]:
df_class['gen_classification'].loc[:10]

0           POSITIVE
1           POSITIVE
2           POSITIVE
3     MIXED FEELINGS
4           POSITIVE
5           POSITIVE
6     MIXED FEELINGS
7           POSITIVE
8     MIXED FEELINGS
9           NEGATIVE
10          NEGATIVE
Name: gen_classification, dtype: object

In [86]:
df_class.head(10)

,profile,anime_uid,text,chunked_text,classifications,mean_score,gen_classification
0,DesolatePsyche,34096,\n \n \n \n ...,[ more pics Overall Story Animation Sound Char...,"[{'label': 'POSITIVE', 'score': 0.969215750694...",0.424159,POSITIVE
1,baekbeans,34599,\n \n \n \n ...,[ more pics Overall Story Animation Sound Char...,"[{'label': 'POSITIVE', 'score': 0.953255474567...",0.797101,POSITIVE
2,skrn,28891,\n \n \n \n ...,[ more pics Overall Story Animation Sound Char...,"[{'label': 'POSITIVE', 'score': 0.994730710983...",0.332255,POSITIVE
3,edgewalker00,2904,\n \n \n \n ...,[ more pics Overall Story Animation Sound Char...,"[{'label': 'NEGATIVE', 'score': 0.999204218387...",-0.022375,MIXED FEELINGS
4,aManOfCulture99,4181,\n \n \n \n ...,[ more pics Overall Story Animation Sound Char...,"[{'label': 'POSITIVE', 'score': 0.997363150119...",0.506765,POSITIVE
5,eneri,2904,\n \n \n \n ...,[ more pics Overall Story Animation Sound Char...,"[{'label': 'POSITIVE', 'score': 0.991586685180...",0.996296,POSITIVE
6,Waffle_Empress,16664,\n \n \n \n ...,[ more pics Overall Story Animation Sound Char...,"[{'label': 'NEGATIVE', 'score': 0.999297976493...",-0.059540,MIXED FEELINGS
7,NIGGER_BONER,2904,\n \n \n \n ...,[ more pics Overall Story Animation Sound Char...,"[{'label': 'NEGATIVE', 'score': 0.995272338390...",0.401125,POSITIVE
8,jchang,2904,\n \n \n \n ...,[ more pics Overall Story Animation Sound Char...,"[{'label': 'POSITIVE', 'score': 0.998441040515...",0.119885,MIXED FEELINGS
9,shadowsplat,4181,\n \n \n \n ...,[ more pics Overall Story Animation Sound Char...,"[{'label': 'POSITIVE', 'score': 0.531278789043...",-0.485066,NEGATIVE


In [ ]:
df_class.drop[['chunked_text', 'profile', ]]

In [ ]:
df_class.to_csv('classed_reviews.csv')

In [7]:
df_class_2 = pd.read_csv('/home/antog96/Jedha_Bootcamp/Projet_DSFS/anime-recommendation-engine/data/reviews_classed.csv')

In [10]:
df_class_2.head()

,profile,anime_uid,text,score,classifications
0,DesolatePsyche,34096,[' more pics Overall 8 Story 8 Animation 8 Sou...,8,"[{'label': 'POSITIVE', 'score': 0.931263029575..."
1,baekbeans,34599,[' more pics Overall 10 Story 10 Animation 10 ...,10,"[{'label': 'POSITIVE', 'score': 0.882984995841..."
2,skrn,28891,[' more pics Overall 7 Story 7 Animation 9 Sou...,7,"[{'label': 'POSITIVE', 'score': 0.972522079944..."
3,edgewalker00,2904,[' more pics Overall 9 Story 9 Animation 9 Sou...,9,"[{'label': 'NEGATIVE', 'score': 0.999156832695..."
4,aManOfCulture99,4181,[' more pics Overall 10 Story 10 Animation 8 S...,10,"[{'label': 'POSITIVE', 'score': 0.998817861080..."


In [8]:
df_class_2["classifications"] = df_class_2["classifications"].apply(ast.literal_eval)

In [9]:
def calculate_mean_score(classifications):
    if isinstance(classifications, list):
        total = 0
        count = 0
        for item in classifications:
            score = item['score']
            if item['label'] == 'NEGATIVE':
                score = -score
            total += score
            count += 1
        return total / count if count else 0
    else:
        return 0

In [11]:
df_class_2.loc[:,'mean_score'] = df_class_2['classifications'].progress_apply(calculate_mean_score)

100%|██████████| 130519/130519 [00:00<00:00, 473757.17it/s]


In [12]:
def determine_sentiment(score):
    if 0.2 <= score <= 1:
        return "POSITIVE"
    elif -0.2 <= score < 0.2:
        return "MIXED FEELINGS"
    else:
        return "NEGATIVE"

In [13]:
df_class_2.loc[:,'gen_classification'] = df_class_2['mean_score'].progress_apply(determine_sentiment)

100%|██████████| 130519/130519 [00:00<00:00, 1054156.97it/s]


In [109]:
df_class_2.head()

,profile,anime_uid,text,score,classifications,mean_score,gen_classification
0,DesolatePsyche,34096,[' more pics Overall 8 Story 8 Animation 8 Sou...,8,"[{'label': 'POSITIVE', 'score': 0.931263029575...",0.418560,POSITIVE
1,baekbeans,34599,[' more pics Overall 10 Story 10 Animation 10 ...,10,"[{'label': 'POSITIVE', 'score': 0.882984995841...",0.573358,POSITIVE
2,skrn,28891,[' more pics Overall 7 Story 7 Animation 9 Sou...,7,"[{'label': 'POSITIVE', 'score': 0.972522079944...",0.306899,POSITIVE
3,edgewalker00,2904,[' more pics Overall 9 Story 9 Animation 9 Sou...,9,"[{'label': 'NEGATIVE', 'score': 0.999156832695...",0.119605,MIXED FEELINGS
4,aManOfCulture99,4181,[' more pics Overall 10 Story 10 Animation 8 S...,10,"[{'label': 'POSITIVE', 'score': 0.998817861080...",0.500251,POSITIVE


In [14]:
df_negative_reviews = df_class_2[df_class_2["gen_classification"] == "NEGATIVE"]
df_positive_reviews = df_class_2[df_class_2["gen_classification"] == "POSITIVE"]
df_mixedfeelings_reviews = df_class_2[df_class_2["gen_classification"] == "MIXED FEELINGS"]

In [15]:
df_negative_reviews.to_csv('df_negative_reviews.csv')
df_positive_reviews.to_csv('df_positive_reviews.csv')
df_mixedfeelings_reviews.to_csv('df_mixedfeelings_reviews.csv')

In [16]:
print(df_negative_reviews.shape)
print(df_positive_reviews.shape)
print(df_mixedfeelings_reviews.shape)

(42270, 7)
(62314, 7)
(25935, 7)


In [17]:
print(df_class_2['anime_uid'].nunique())
print(f'Animes with negative reviews : {df_negative_reviews['anime_uid'].nunique()}')
print(f'Animes with positive reviews : {df_positive_reviews['anime_uid'].nunique()}')
print(f'Animes with mixed reviews : {df_mixedfeelings_reviews['anime_uid'].nunique()}')

8113
Animes with negative reviews : 5799
Animes with positive reviews : 5920
Animes with mixed reviews : 4678


In [18]:
animes_with_neg_reviews = set(df_negative_reviews['anime_uid'].unique())
animes_with_pos_reviews = set(df_positive_reviews['anime_uid'].unique())
animes_with_mixed_reviews = set(df_mixedfeelings_reviews['anime_uid'].unique())

In [130]:
animes_only_in_mixed = (animes_with_mixed_reviews - animes_with_neg_reviews) - animes_with_pos_reviews
len(animes_only_in_mixed)

529

In [131]:
animes_only_in_neg = (animes_with_neg_reviews - animes_with_pos_reviews) - animes_with_mixed_reviews
len(animes_only_in_neg)

1198

In [133]:
animes_only_in_pos = (animes_with_pos_reviews - animes_with_neg_reviews) - animes_with_mixed_reviews
len(animes_only_in_pos)

1247

In [21]:
def unchunk_text(text):
    fused_text = ""
    for chunk in text:
        fused_text += chunk
    return fused_text


In [19]:
df_negative_reviews['text'] = df_negative_reviews['text'].apply(ast.literal_eval)
df_positive_reviews['text'] = df_positive_reviews['text'].apply(ast.literal_eval)
df_mixedfeelings_reviews['text'] = df_mixedfeelings_reviews['text'].apply(ast.literal_eval)

/tmp/ipykernel_1053/991039155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negative_reviews['text'] = df_negative_reviews['text'].apply(ast.literal_eval)
/tmp/ipykernel_1053/991039155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positive_reviews['text'] = df_positive_reviews['text'].apply(ast.literal_eval)
/tmp/ipykernel_1053/991039155.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [22]:
df_negative_reviews['text'] = df_negative_reviews['text'].apply(unchunk_text)
df_positive_reviews['text'] = df_positive_reviews['text'].apply(unchunk_text)
df_mixedfeelings_reviews['text'] = df_mixedfeelings_reviews['text'].apply(unchunk_text)

/tmp/ipykernel_1053/2633916163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negative_reviews['text'] = df_negative_reviews['text'].apply(unchunk_text)
/tmp/ipykernel_1053/2633916163.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positive_reviews['text'] = df_positive_reviews['text'].apply(unchunk_text)
/tmp/ipykernel_1053/2633916163.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [23]:
def split_text_for_summary(text):    
    text_splited = re.findall('.{1024}', text)
    text_splited.append(text[1024 * len(text_splited):])
    return text_splited

In [24]:
df_negative_reviews.loc[:,'chunked_text'] = df_negative_reviews.loc[:,"text"].progress_apply(split_text_for_summary)
df_positive_reviews.loc[:,'chunked_text'] = df_positive_reviews.loc[:,"text"].progress_apply(split_text_for_summary)
df_mixedfeelings_reviews.loc[:,'chunked_text'] = df_mixedfeelings_reviews.loc[:,"text"].progress_apply(split_text_for_summary)

100%|██████████| 42270/42270 [00:00<00:00, 232760.84it/s]
/tmp/ipykernel_1053/3806439965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negative_reviews.loc[:,'chunked_text'] = df_negative_reviews.loc[:,"text"].progress_apply(split_text_for_summary)
100%|██████████| 62314/62314 [00:00<00:00, 204367.58it/s]
/tmp/ipykernel_1053/3806439965.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positive_reviews.loc[:,'chunked_text'] = df_positive_reviews.loc[:,"text"].progress_apply(split_text_for_summa

In [54]:
display(df_negative_reviews.head(8))
pd.set_option('display.max_colwidth', None)
display(df_negative_reviews.loc[:8,'chunked_text'])

,profile,anime_uid,text,score,classifications,mean_score,gen_classification,chunked_text
6,Waffle_Empress,16664,more pics Overall 6 Story 6 Animation 9 Sound...,6,"[{'label': 'NEGATIVE', 'score': 0.999612748622...",-0.672079,NEGATIVE,[ more pics Overall 6 Story 6 Animation 9 Soun...
9,shadowsplat,4181,more pics Overall 4 Story 8 Animation 9 Sound...,4,"[{'label': 'POSITIVE', 'score': 0.621118187904...",-0.458163,NEGATIVE,[ more pics Overall 4 Story 8 Animation 9 Soun...
10,angelsreview,4672,more pics Overall 8 Story 6 Animation 7 Sound...,8,"[{'label': 'NEGATIVE', 'score': 0.999028086662...",-0.385837,NEGATIVE,[ more pics Overall 8 Story 6 Animation 7 Soun...
20,Rigas,4181,more pics Overall 5 Story 4 Animation 7 Sound...,5,"[{'label': 'NEGATIVE', 'score': 0.967860877513...",-0.652592,NEGATIVE,[ more pics Overall 5 Story 4 Animation 7 Soun...
24,Leyvon,4181,more pics Overall 10 Story 10 Animation 10 So...,10,"[{'label': 'NEGATIVE', 'score': 0.990030050277...",-0.990030,NEGATIVE,[ more pics Overall 10 Story 10 Animation 10 S...
30,SieghartTheMage,1253,more pics Overall 8 Story 8 Animation 8 Sound...,8,"[{'label': 'NEGATIVE', 'score': 0.999048769474...",-0.997749,NEGATIVE,[ more pics Overall 8 Story 8 Animation 8 Soun...
31,ggultra2764,1425,more pics Overall 7 Story 7 Animation 6 Sound...,7,"[{'label': 'NEGATIVE', 'score': 0.992316722869...",-0.252149,NEGATIVE,[ more pics Overall 7 Story 7 Animation 6 Soun...
35,gwern,7588,more pics Overall 7 Story 7 Animation 6 Sound...,7,"[{'label': 'NEGATIVE', 'score': 0.999524593353...",-0.999345,NEGATIVE,[ more pics Overall 7 Story 7 Animation 6 Soun...


6    [ more pics Overall 6 Story 6 Animation 9 Sound 8 Character 6 Enjoyment 6 As someone who loves Studio Ghibli and its movies I expected to see something incredible and life changing However this movie was kind of a disappointment as I found it hard to remain attentive or just care for the characters or the plot Story 6 10 I noticed that I am not really interested in adaptations of legends and myths despite my interest in them as well as cultures of many different cultures Japan being one of them It kind of reminds me of Thumbelina to an extent although I am fully aware that the Japanese fairy tale is way older than its counterpart from the 19th century please don t get me wrong It s a story about a mysterious girl so beautiful that even princes were dying to ask for her hand in marriage However Kaguya s destiny has nothing to do with ordinary human life even that of a noble It s slow and overly complicated which normally works in other Ghibli movies but this time these unnecessary 

In [ ]:
df_negative_reviews.loc[:,"summaries"] = pipe_summarization()

100%|██████████| 3/3 [01:08<00:00, 22.99s/it]
/tmp/ipykernel_1053/1117716840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negative_reviews.loc[:,"summaries"] = df_negative_reviews.loc[:10,'text'].progress_apply(pipe_summarization)


In [63]:
def extract_summary_text(element):
    if isinstance(element, dict):
        return element.get('summary_text', '')
    elif isinstance(element, list):
        # Si c'est une liste, essayez de prendre le premier élément ou une chaîne vide
        return element[0].get('summary_text', '') if element and isinstance(element[0], dict) else ''
    return ''

In [64]:
df_negative_reviews['summary'] = df_negative_reviews['summaries'].apply(extract_summary_text)

/tmp/ipykernel_1053/536896796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negative_reviews['summary'] = df_negative_reviews['summaries'].apply(extract_summary_text)


In [89]:
df_negative_reviews = df_negative_reviews["summary"].apply(ast.literal_eval)

SyntaxError: invalid syntax (<unknown>, line 1)

In [81]:
type(df_negative_reviews.loc[:8,'summary'])

pandas.core.series.Series

In [57]:
question_answerer = pipeline("question-answering", model='distilbert-base-uncased-distilled-squad')

Device set to use cuda:0


In [84]:
questions = "this is a text for test 1"

In [ ]:
question_answerer(question="What are the positive and negative points expressed in the context text ?", context=)

{'score': 0.8598906397819519, 'start': 19, 'end': 25, 'answer': 'test 1'}

In [93]:
mistral = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.3")

Loading checkpoint shards:  33%|███▎      | 1/3 [00:10<00:20, 10.10s/it]

: 

In [91]:
from huggingface_hub import login

In [ ]:
message = ""

In [88]:
messages = [
    {"role": "user", "content": f'Read the summary of the reviews in {message} and extract the 2 most important drawbacks expressed'}
]

NameError: name 'message' is not defined